# Wave Function Optimization

We present here a complete example on how to use QMCTorch on a H2 molecule.
We first need to import all the relevant modules :

In [2]:
from torch import optim
from qmctorch.scf import Molecule
from qmctorch.wavefunction import SlaterJastrow
from qmctorch.solver import SolverSlaterJastrow
from qmctorch.sampler import Metropolis
from qmctorch.utils import set_torch_double_precision
from qmctorch.utils import (plot_energy, plot_data)
set_torch_double_precision()

## Creating the system

The first step is to define a molecule. We here use a H2 molecule with both hydrgen atoms
on the z-axis and separated by 1.38 atomic unit. We choose here to use Slater orbitals that can be otained via `ADF`. We simply here reload calculations to create the molecule

In [3]:
mol = Molecule(load='./hdf5/H2_adf_dzp.hdf5')

INFO:QMCTorch|
INFO:QMCTorch| SCF Calculation
INFO:QMCTorch|  Loading data from H2_adf_dzp.hdf5


We then define the wave function relative to this molecule. We also specify here
the determinants we want to use in the CI expansion. We use here a to include all the single
and double excitation with 2 electrons and 2 orbitals

In [5]:
wf = SlaterJastrow(mol, configs='single_double(2,2)')

INFO:QMCTorch|
INFO:QMCTorch| Wave Function
INFO:QMCTorch|  Jastrow factor      : True
INFO:QMCTorch|  Jastrow kernel      : PadeJastrowKernel
INFO:QMCTorch|  Highest MO included : 10
INFO:QMCTorch|  Configurations      : single_double(2,2)
INFO:QMCTorch|  Number of confs     : 4
INFO:QMCTorch|  Kinetic energy      : jacobi
INFO:QMCTorch|  Number var  param   : 121
INFO:QMCTorch|  Cuda support        : False


As a sampler we use a simple Metropolis Hasting with 1000 walkers. The walkers are initially localized around the atoms.
Each walker will perform 2000 steps of size 0.2 atomic unit and will only keep the last position of each walker (`ntherm=-1`).
During each move all the the electrons are moved simultaneously within a normal distribution centered around their current location.

In [6]:
sampler = Metropolis(nwalkers=1000,
                     nstep=2000, step_size=0.2,
                     ntherm=-1, ndecor=100,
                     nelec=wf.nelec, init=mol.domain('atomic'),
                     move={'type': 'all-elec', 'proba': 'normal'})

INFO:QMCTorch|
INFO:QMCTorch| Monte-Carlo Sampler
INFO:QMCTorch|  Number of walkers   : 1000
INFO:QMCTorch|  Number of steps     : 2000
INFO:QMCTorch|  Step size           : 0.2
INFO:QMCTorch|  Thermalization steps: -1
INFO:QMCTorch|  Decorelation steps  : 100
INFO:QMCTorch|  Walkers init pos    : atomic
INFO:QMCTorch|  Move type           : all-elec
INFO:QMCTorch|  Move proba          : normal


We will use the ADAM optimizer implemented in pytorch with custom learning rate for each layer.
We also define a linear scheduler that will decrease the learning rate after 100 steps

In [7]:
lr_dict = [{'params': wf.jastrow.parameters(), 'lr': 1E-3},
           {'params': wf.ao.parameters(), 'lr': 1E-6},
           {'params': wf.mo.parameters(), 'lr': 1E-3},
           {'params': wf.fc.parameters(), 'lr': 1E-3}]
opt = optim.Adam(lr_dict, lr=1E-3)

A scheduler can also be used to progressively decrease the value of the learning rate during the optimization.

In [9]:
scheduler = optim.lr_scheduler.StepLR(opt, step_size=100, gamma=0.90)

We can now assemble the solver

In [10]:
solver = SolverSlaterJastrow(wf=wf, sampler=sampler, optimizer=opt, scheduler=None)

INFO:QMCTorch|
INFO:QMCTorch| QMC Solver 
INFO:QMCTorch|  WaveFunction        : SlaterJastrow
INFO:QMCTorch|  Sampler             : Metropolis
INFO:QMCTorch|  Optimizer           : Adam


## Comfiguring the solver

Many parameters of the optimization can be controlled. We can specify which observale to track during the optimization. Here only the local energies will be recorded but one can also record the variational parameters

In [11]:
solver.configure(track=['local_energy'])

Some variational parameters can be frozen and therefore not optimized. We here freeze the MO coefficients and the AO parameters
and therefore only the jastrow parametres and the CI coefficients will be optmized

In [12]:
solver.configure(freeze=['ao', 'mo'])

Either the mean or the variance of local energies can be used as a loss function. We choose here to minimize the energy to optimize the wave function

In [13]:
solver.configure(loss='energy')

The gradients of the wave function w.r.t. the variational parameters can be computed directly via automatic differntiation (`grad='auto'`)or manually (`grad='auto'`) via a reduced noise formula. We pick here a manual calculation

In [14]:
solver.configure(grad='manual')

We also configure the resampling so that the positions of the walkers are updated by performing
25 MC steps from their previous positions after each optimization step.

In [15]:
solver.configure(resampling={'mode': 'update',
                            'resample_every': 1,
                            'nstep_update': 25})

## Running the wave function optimization

We can now run the optimization. We use here 250 optimization steps (epoch), using all the points
in a single mini-batch.

In [16]:
solver.run(50)

INFO:QMCTorch|
INFO:QMCTorch|  Optimization
INFO:QMCTorch|  Task                :
INFO:QMCTorch|  Number Parameters   : 115
INFO:QMCTorch|  Number of epoch     : 250
INFO:QMCTorch|  Batch size          : 1000
INFO:QMCTorch|  Loss function       : energy
INFO:QMCTorch|  Clip Loss           : False
INFO:QMCTorch|  Gradients           : manual
INFO:QMCTorch|  Resampling mode     : update
INFO:QMCTorch|  Resampling every    : 1
INFO:QMCTorch|  Resampling steps    : 25
INFO:QMCTorch|  Output file         : H2_adf_dzp_QMCTorch.hdf5
INFO:QMCTorch|  Checkpoint every    : None
INFO:QMCTorch|
INFO:QMCTorch|
INFO:QMCTorch|  epoch 0
INFO:QMCTorch|  energy   : -1.157424 +/- 0.007778
INFO:QMCTorch|  variance : 0.245959
INFO:QMCTorch|  epoch done in 0.15 sec.
INFO:QMCTorch|
INFO:QMCTorch|  epoch 1
INFO:QMCTorch|  energy   : -1.144750 +/- 0.007154
INFO:QMCTorch|  variance : 0.226236
INFO:QMCTorch|  epoch done in 0.17 sec.
INFO:QMCTorch|
INFO:QMCTorch|  epoch 2
INFO:QMCTorch|  energy   : -1.147760 +/- 

KeyboardInterrupt: 